In [ ]:
import csv
import time

from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://siam-shop.ru/catalog/tayskaya_kukhnya/")

In [2]:
catalog_name, catalog_price = [],[]
# собираем информацию со странички каталога, вычленяем название, цену товара,
# переходим на его страницу и там берем описание
# данные о товарах складываем в общий список

def parse_data(data, class_names):
    for product in data:
        name = product.find_element_by_class_name(class_names["name"]).text
        price = product.find_element_by_class_name(class_names["price"]).text
        catalog_name.append(name)
        catalog_price.append(price)

urls =[]
base_url = "https://siam-shop.ru"
data = driver.find_elements_by_class_name('b-catalog-sections_item_holder')
parse_data(data, {"name":"b-catalog-sections_item_title", "price": "bx_price"})
response = requests.get(driver.current_url)
soup = BeautifulSoup(response.text, "lxml")
names = soup.find_all('div', class_="b-catalog-sections_item_title")

for name in names:
    urls.append(base_url+name.a['href'])

#print(urls)

In [3]:
import threading
import sys

descrip=[]
def thread_job(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        descrip.append(soup.find_all('div', class_="b-catalog-detail_detail-text")[0].text)
    except BaseException:
        print(url, soup.find_all('div', class_="b-catalog-detail_detail-text"))

def run_threads(count):
    threads_list = []

    for street in urls:
        get_description = threading.Thread(
            target=thread_job, args=(street,)
        )
        get_description.start()
        threads_list.append(get_description)

    for t in threads_list:
        t.join()

run_threads(3)

In [4]:
# запись в формат csv
catalog_list = pd.DataFrame({'Name': catalog_name, 'Price': catalog_price, 'Description': descrip})
catalog_list.to_csv('catalog_list.csv',index=False)
    
with open("catalog_list.csv", 'r', encoding="utf-8") as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        print(row)

In [ ]:
driver.close()